In [1]:
! wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
! tar -xvzf ml-20m-compact.tar.gz
! pip install caserecommender

--2022-05-06 13:39:48--  https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz [following]
--2022-05-06 13:39:48--  https://raw.githubusercontent.com/mmanzato/MBABigData/master/ml-20m-compact.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 65019041 (62M) [application/octet-stream]
Saving to: ‘ml-20m-compact.tar.gz’

ml-20m-compact.tar. 100%[===================>]  62.01M   261MB/s    in 0.2s    

2022-05-06 13:39:50 (261 MB/s) - ‘ml-20m-compact.tar.gz’ saved [65019041/6501

In [2]:
import numpy as np
import pandas as pd

# Ler e preparar dados (vide notebook anterior)


In [3]:
from sklearn.model_selection import train_test_split

movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title

In [4]:
test = pd.read_csv('test.dat', sep='\t', names=['userId', 'movieId', 
                                                'rating', 'title'])
test[test.userId == 0]

,userId,movieId,rating,title
11794,0,0,5.0,Enemy Mine (1985)
23603,0,5,5.0,"I, Robot (2004)"


## Memory-based

In [5]:
from caserec.recommenders.rating_prediction.itemknn import ItemKNN

ItemKNN('train.dat', 'test.dat', 'rp_iknn.dat', as_similar_first=True).compute()

[Case Recommender: Rating Prediction > ItemKNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 7.291614 sec
prediction_time:: 2.745314 sec
Eval:: MAE: 0.665964 RMSE: 0.876739 


In [6]:
predictions = pd.read_csv('rp_iknn.dat', sep='\t', names=['userId', 'movieId', 'rating'])
predictions['title'] = predictions.movieId.map(map_title)
predictions.head(3)

,userId,movieId,rating,title
0,0,0,4.263840,Enemy Mine (1985)
1,0,5,3.824934,"I, Robot (2004)"
2,1,6,3.511018,Apocalypto (2006)


In [7]:
from caserec.recommenders.rating_prediction.userknn import UserKNN

UserKNN('train.dat', 'test.dat', 'rp_uknn.dat').compute()

[Case Recommender: Rating Prediction > UserKNN Algorithm]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 8.412465 sec
prediction_time:: 292.142221 sec
Eval:: MAE: 0.656318 RMSE: 0.845554 


In [8]:
predictions = pd.read_csv('rp_uknn.dat', sep='\t', names=['user_id', 'movieId', 'rating'])
predictions['title'] = predictions.movieId.map(map_title)
predictions.head(3)

,user_id,movieId,rating,title
0,0,0,3.915260,Enemy Mine (1985)
1,0,5,3.694669,"I, Robot (2004)"
2,1,6,3.587327,Apocalypto (2006)


## Model-based

In [9]:
from caserec.recommenders.rating_prediction.matrixfactorization import MatrixFactorization

MatrixFactorization('train.dat', 'test.dat', 'rp_mf.dat', factors=3).compute()

[Case Recommender: Rating Prediction > Matrix Factorization]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 40.377100 sec
prediction_time:: 0.130994 sec


Eval:: MAE: 0.659785 RMSE: 0.867101 


In [10]:
from caserec.recommenders.rating_prediction.svd import SVD

SVD('train.dat', 'test.dat', 'rp_mf.dat', factors=30).compute()

[Case Recommender: Rating Prediction > SVD]

train data:: 11090 users and 403 items (152496 interactions) | sparsity:: 96.59%
test data:: 10503 users and 340 items (38125 interactions) | sparsity:: 98.93%

training_time:: 1.123805 sec
prediction_time:: 0.122138 sec


Eval:: MAE: 0.916023 RMSE: 1.161923 
